# Statistical analysis made easy in Python with SciPy and pandas DataFrames

http://www.randalolson.com/2012/08/06/statistical-analysis-made-easy-in-python/

In this section, we introduce a few useful methods for analyzing your data in Python. Namely, we cover how to compute the mean, variance, and standard error of a data set. For more advanced statistical analysis, we cover how to perform a Mann-Whitney-Wilcoxon (MWW) RankSum test, how to perform an Analysis of variance (ANOVA) between multiple data sets, and how to compute bootstrapped 95% confidence intervals for non-normally distributed data sets.

For the purposes of this tutorial, we will use Luis Zaman’s [digital parasite data set](http://labfab.cc/).

In [1]:
import pandas as pd

experiment_df = pd.read_csv("datasets/parasite_data.csv", na_values=[" "])

In [2]:
experiment_df.head()

,Virulence,Replicate,ShannonDiversity
0,0.5,1,0.059262
1,0.5,2,1.093600
2,0.5,3,1.139390
3,0.5,4,0.547651
4,0.5,5,0.065928


In [3]:
experiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 3 columns):
Virulence           300 non-null float64
Replicate           350 non-null int64
ShannonDiversity    350 non-null float64
dtypes: float64(2), int64(1)
memory usage: 8.3 KB


We need to take care of NaN value before moving on. We can either drop NaN values or fill them in with some value.

In [4]:
experiment_drop_df = experiment_df.dropna()

In [5]:
experiment_fill_df = experiment_df.fillna(0.0)

In [6]:
experiment_drop_df["Virulence"].mean()

0.7500000000000013

In [7]:
experiment_fill_df["Virulence"].mean()

0.642857142857144

### Mean of a data set

The mean performance of an experiment gives a good idea of how the experiment will turn out on average under a given treatment.

In [10]:
print("Mean Shannon Diversity w/ 0.8 Parasite Virulence =",  
      experiment_df[experiment_df["Virulence"] == 0.8]["ShannonDiversity"].mean())  

Mean Shannon Diversity w/ 0.8 Parasite Virulence = 1.2691338187999996


### Variance in a data set
The variance in the performance provides a measurement of how consistent the results of an experiment are. The lower the variance, the more consistent the results are, and vice versa.



In [13]:
print ("Variance in Shannon Diversity w/ 0.8 Parasite Virulence =",  
       experiment_df[experiment_df["Virulence"] == 0.8]["ShannonDiversity"].var())  

Variance in Shannon Diversity w/ 0.8 Parasite Virulence = 0.6110384333126732


### Standard Error of the Mean (SEM)
Combined with the mean, the SEM enables you to establish a range around a mean that the majority of any future replicate experiments will most likely fall within.

pandas DataFrames don’t have methods like SEM built in, but since DataFrame rows/columns are treated as lists, you can use any NumPy/SciPy method you like on them.

In [14]:
from scipy import stats

In [15]:
print ("SEM of Shannon Diversity w/ 0.8 Parasite Virulence =",  
    stats.sem(experiment_df[experiment_df["Virulence"] == 0.8]["ShannonDiversity"])) 

SEM of Shannon Diversity w/ 0.8 Parasite Virulence = 0.110547585529


A single SEM will usually envelop 68% of the possible replicate means and two SEMs envelop 95% of the possible replicate means. Two SEMs are called the “estimated 95% confidence interval.” The confidence interval is estimated because the exact width depend on how many replicates you have; this approximation is good when you have more than 20 replicates.

### Mann-Whitney-Wilcoxon (MWW) RankSum test

The MWW RankSum test is a useful test to determine if two distributions are significantly different or not. Unlike the t-test, the RankSum test does not assume that the data are normally distributed, potentially providing a more accurate assessment of the data sets.

As an example, let’s say we want to determine if the results of the two following treatments significantly differ or not.

In [17]:
treatment1 = experiment_df[experiment_df["Virulence"] == 0.5]["ShannonDiversity"]
treatment2 = experiment_df[experiment_df["Virulence"] == 0.8]["ShannonDiversity"]

In [18]:
treatment1

0     0.059262
1     1.093600
2     1.139390
3     0.547651
4     0.065928
5     1.344330
6     1.680480
7     0.000000
8     2.047680
9     0.000000
10    1.507140
11    0.000000
12    1.589810
13    1.144800
14    1.011190
15    0.000000
16    0.776665
17    0.001749
18    1.761200
19    0.021091
20    0.790915
21    0.000000
22    0.018866
23    0.994268
24    1.729620
25    0.967537
26    0.457318
27    0.992525
28    1.506640
29    0.697241
30    1.790580
31    1.787710
32    0.857742
33    0.000000
34    0.445267
35    0.045471
36    0.003490
37    0.000000
38    0.115830
39    0.980076
40    0.000000
41    0.820405
42    0.124755
43    0.719755
44    0.584252
45    1.937930
46    1.284150
47    1.651680
48    0.000000
49    0.000000
Name: ShannonDiversity, dtype: float64

In [19]:
treatment2

150    1.433800
151    2.079700
152    0.892139
153    2.384740
154    0.006980
155    1.971760
156    0.000000
157    1.428470
158    1.715950
159    0.000000
160    0.421927
161    1.179920
162    0.932470
163    2.032520
164    0.960912
165    2.384150
166    1.879130
167    1.238890
168    1.584300
169    1.118490
170    2.022970
171    0.000000
172    2.138820
173    2.533390
174    1.212340
175    0.059135
176    1.578260
177    1.725210
178    0.293153
179    0.000000
180    0.000000
181    1.699600
182    2.178650
183    1.792580
184    1.920800
185    0.000000
186    1.583250
187    0.343235
188    1.980010
189    0.980876
190    1.089380
191    0.979254
192    1.190450
193    1.738880
194    1.154100
195    1.981610
196    2.077180
197    1.566410
198    0.000000
199    1.990900
Name: ShannonDiversity, dtype: float64

A RankSum test will provide a P value indicating whether or not the two distributions are the same.

In [20]:
stats.ranksums(treatment1, treatment2)

RanksumsResult(statistic=-3.2952459004684997, pvalue=0.00098335590273505771)

If P <= 0.05, we are highly confident that the distributions significantly differ, and can claim that the treatments had a significant impact on the measured value. 

If the treatments do not significantly differ, we could expect a result such as the following.

In [23]:
treatment3 = experiment_df[experiment_df["Virulence"] == 0.8]["ShannonDiversity"]
treatment4 = experiment_df[experiment_df["Virulence"] == 0.9]["ShannonDiversity"]

In [25]:
stats.ranksums(treatment3, treatment4)

RanksumsResult(statistic=0.0068938198754571129, pvalue=0.99449957112420484)

With P > 0.05, we must say that the distributions do not significantly differ. Thus changing the parasite virulence between 0.8 and 0.9 does not result in a significant change in Shannon Diversity.

### One-way analysis of variance (ANOVA)

If you need to compare more than two data sets at a time, an ANOVA is your best bet. For example, we have the results from three experiments with overlapping 95% confidence intervals, and we want to confirm that the results for all three experiments are not significantly different.

In [27]:
treatment1 = experiment_df[experiment_df["Virulence"] == 0.7]["ShannonDiversity"]
treatment2 = experiment_df[experiment_df["Virulence"] == 0.8]["ShannonDiversity"]
treatment3 = experiment_df[experiment_df["Virulence"] == 0.9]["ShannonDiversity"]

In [28]:
stats.f_oneway(treatment1, treatment2, treatment3)

F_onewayResult(statistic=0.96996402324598563, pvalue=0.38150948187410261)

If P > 0.05, we can claim with high confidence that the means of the results of all three experiments are not significantly different.